In [16]:
import os, sys
import argparse
import time

import xml.etree.ElementTree as ET

from sumolib import checkBinary

import traci
import traci.constants as tc

In [17]:
from routeGenerate import GenerateRoutes 

In [18]:
import curbside

In [19]:
net_xml = "seattle.net.xml"
add_xml = "seattle.add.xml"
rou_xml = "seattle.trips.xml"

In [20]:
def simulate(seconds = 3600, gui = False):
    """
    Main function that controls the simulation
    """

    road_network = curbside.create_graph('seattle.net.xml')
    
    # create agent curbs
    curb_ids = []
    root = ET.parse(add_xml).getroot()
    for child in root.iter('additional'):
        for kid in child.iter('parkingArea'):
            curb_ids.append(kid.get('id'))
    
    curbs = []
    for curb_id in curb_ids:
        curbs.append(curbside.SmartCurbside(1, 'seattle.add.xml', 'seattle.net.xml', curb_id, 'passenger', road_network))
    
    start = time.time()
    
    reroute_num = 0

    if gui:
        sumoBinary = checkBinary('sumo-gui')
    else:
        sumoBinary = checkBinary('sumo')

    traci.start([sumoBinary, "-c", "seattle.sumocfg"])
    
    # simulation
    while traci.simulation.getMinExpectedNumber() > 0:
        traci.simulationStep()
        if traci.simulation.getTime() == seconds:
            running_time = time.time() - start
#             print(running_time)
            break
        
        for curb in curbs:
            # check if there is any vehicle enters
            v_enter = set(traci.edge.getLastStepVehicleIDs(curb.edge)) - curb.moving_vehicle
            # v_enter -= v_leave
            
            for veh in v_enter:
                # new vehicle is for sure not confirmed
                if traci.vehicle.getNextStops(veh) and curb.id in [item[2] for item in traci.vehicle.getNextStops(veh)] and not traci.vehicle.isStopped(veh):
                    # if the trips is not ending and the current edge is the parking stop
                    # v_leave excluded
                    if traci.vehicle.getVehicleClass(veh) == curb.vclass and len(curb.occupied_vehicle) < curb.capacity:
                        # if can park
                        curb.occupied_vehicle.add(veh)
                    
                    else:
                        # cannot park at this edge, reroute
                        # item._reroute_choice() returns (curb_id, distance) tuple
                        traci.vehicle.rerouteParkingArea(veh, curb._reroute_choice()[0])
                        reroute_num += 1
                        
            v_leave = curb.parked_vehicle - set(traci.parkingarea.getVehicleIDs(curb.id))
            curb.occupied_vehicle -= v_leave
            curb.parked_vehicle = set(traci.parkingarea.getVehicleIDs(curb.id))
            curb.moving_vehicle = set(traci.edge.getLastStepVehicleIDs(curb.edge))
            curb._update_nearby_curb()

    # enter False to disconnect before SUMO finishes
    traci.close()

    # flush out intermmediate results
    sys.stdout.flush()
    
    return reroute_num, running_time


In [9]:
demand_factors = [1.0, 1.5, 2.0, 2.5, 3]
# demand_factors = [3]

In [10]:
GenerateRoutes(demand_factor = 3.0, simulate_hour = 12)

seattle.trips.xml re-generated. (demand factor = 3.0 )


In [14]:
reroute_nums = []
for demand_factor in demand_factors:
    GenerateRoutes(demand_factor, simulate_hour = 12)
    reroute_num, running_time = simulate(seconds = 1000, gui = False)
    print('demand factor: {}, # of reroute: {}, running time: {}'.format(demand_factor, reroute_num, round(running_time,1)))
    reroute_nums.append(reroute_num)
    

seattle.trips.xml re-generated. (demand factor = 1.0 )
demand factor: 1.0, # of reroute: 3, running time: 40.5
seattle.trips.xml re-generated. (demand factor = 1.5 )
demand factor: 1.5, # of reroute: 9, running time: 38.2
seattle.trips.xml re-generated. (demand factor = 2.0 )
demand factor: 2.0, # of reroute: 10, running time: 40.1
seattle.trips.xml re-generated. (demand factor = 2.5 )
demand factor: 2.5, # of reroute: 12, running time: 41.3
seattle.trips.xml re-generated. (demand factor = 3 )
demand factor: 3, # of reroute: 20, running time: 41.1


In [13]:
traci.close()